In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from time import sleep
import random
import datetime, threading, time
#downloaded_page_number = '82487'

In [2]:
file_name = 'mfd_sber_downloaded.csv'
user_agent = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url = 'http://forum.mfd.ru/forum/thread/?id=62075&page={}'

In [50]:
def get_html(url):
    r = requests.get(url, headers=user_agent)
    return r.text

def get_last_page_number():
    last_page = 'http://forum.mfd.ru/forum/thread/?id=62075&page='
    try:
        html = get_html(last_page)
        soup = BeautifulSoup(html, 'lxml')
        paginator = soup.find('div', class_="mfd-paginator")
        last_page_number = paginator.find_all('a')[-1].text
    except:
        print('CANT GET LAST PAGE NUMBER, SOME ERROR')
        raise TypeError
    return last_page_number

def get_page_data(url, writer):
    try:
        html = get_html(url)
        soup = BeautifulSoup(html, 'lxml')
        comments = soup.find_all('div', class_="mfd-post")
        for comment in comments:
            try:
                name = comment.find('div', class_='mfd-post-top-0').text
                ct = comment.find('div', class_='mfd-post-text')
                if 'blockquote' in str(ct):
                    try:
                        ct.find("blockquote").extract()
                    except:
                        pass
                text = ct.text.strip()
                date = comment.find('div', class_='mfd-post-top-1').text[:10]
                data = {'name': name,
                            'text': text,
                            'date': date,
                            'url': url}
                writer.writerow(data)
            except:
                #print(comment.text)
                pass
    except:
        sleep(10)
        print(url)
        pass

In [61]:
def make_daily_download():
    last_page_number = get_last_page_number()
    with open(file_name, "r") as file:
        str1 = file.readline()
        downloaded_page_number = str1.split()[-1].split('=')[-1]
    if int(last_page_number)-1 > int(downloaded_page_number):
        with open(file_name, 'a') as f:
            order = ['name', 'text', 'date', 'url']
            writer = csv.DictWriter(f, fieldnames=order)
            for i in range(int(last_page_number)-1, int(downloaded_page_number), -1):
                get_page_data(url.format(str(i)), writer)
                sleep(random.randrange(1,3))
        print('TEST: work done, FILE CLOSED')